In [1]:
import time

from model import WaveNetModel, Optimizer, WaveNetData

from IPython.display import Audio
from IPython.core.debugger import Tracer
from matplotlib import pyplot as plt
from matplotlib import pylab as pl
from IPython import display
import torch
import numpy as np
#%matplotlib inline
%matplotlib notebook

In [2]:
model = WaveNetModel(num_layers=10,
                     num_blocks=2,
                     num_classes=128,
                     hidden_channels=64)

model.load_state_dict(torch.load("model_parameters/violin_10-2-128-164"))

data = WaveNetData('train_samples/violin.wav',
                   input_length=model.scope,
                   target_length=model.last_block_scope,
                   num_classes=model.num_classes)

In [15]:
start_tensor = data.get_minibatch([model.scope])[0].squeeze()
#start_tensor = torch.zeros((model.scope)) + 0.0

plt.ion()
plt.plot(start_tensor.numpy())
plt.ioff()

In [8]:
print('generate...')
tic = time.time()
#generated = model.generate(start_data=start_tensor, num_generate=100)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

fig = plt.figure()
plt.plot(np.asarray(generated[-100:]))

generate...
Generating took 2.384185791015625e-05 seconds.


In [4]:
print('generate...')
tic = time.time()
model.sample_generation = False
#[fast_generated, support_generated] = model.fast_generate(600, first_samples=torch.zeros((1)))
[fast_generated, support_generated] = model.fast_generate(600, first_samples=start_tensor)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

fig = plt.figure()
plt.plot(fast_generated)

generate...
Generating took 20.40074896812439 seconds.


In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display
progress = FloatProgress(min=0, max=100)
display(progress)

def p_callback(i, total):
    progress.value += 1

sample_length = 44100

tic = time.time()
#[generated_sample, support_generated] = model.fast_generate(sample_length, first_samples=torch.zeros((1))+0.1)
generated_sample = model.fast_generate(sample_length, 
                                       first_samples=start_tensor,
                                       #first_samples=torch.zeros((1)),
                                       progress_callback=p_callback)
toc = time.time()
print('Generating took {} seconds.'.format(toc-tic))

In [29]:
np_sample = np.asarray(generated_sample)
print("sample: ", np_sample)

from scipy.io import wavfile
wavfile.write('generated_bach_11025_sample_12_layers.wav', 11025, np_sample)

sample:  [ 0.1484375  0.125      0.0234375 ...,  0.2421875  0.140625  -0.0390625]


In [14]:
fig = plt.figure()
plt.plot(generated_sample[0:1000])

from IPython.display import Audio
Audio(generated_sample, rate=44100)